In [1]:
import os

import pandas as pd
import autonormalize as an

In [2]:
crime_df = pd.read_csv(os.path.join(os.getcwd(), 'autonormalize/downloads/crime.csv'))
crime_df = crime_df.drop(columns=crime_df.columns[10:])
crime_df = crime_df.dropna()
crime_df = crime_df.drop_duplicates()
print("Rows: "+ str(crime_df.shape[0]))
print("Columns: " + str(crime_df.shape[1]))
crime_df.head(3)

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Rows: 638454
Columns: 10


,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type
0,1,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,January,1,Murder or Manslaughter
1,2,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,1,Murder or Manslaughter
2,3,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,2,Murder or Manslaughter


In [5]:
deps_exact = an.find_dependencies(crime_df, 1.00)
deps_approx = an.find_dependencies(crime_df, 0.94)
print("Exact dependencies...")
print(deps_exact)
print("\nApproximate dependencies...")
print(deps_approx)

100%|██████████| 9/9 [11:11<00:00, 42.99s/it] 


Exact dependencies...
 --> Record ID
 {Record ID}  --> Agency Code
 {Record ID}  --> Agency Name
 {Record ID}  --> Agency Type
 {Record ID}  --> City
 {Record ID}  {Agency Code}  --> State
 {Record ID}  --> Year
 {Record ID}  --> Month
 {Record ID}  --> Incident
 {Record ID}  --> Crime Type

Approximate dependencies...
 --> Record ID
 {Record ID}  {Agency Name,City}  {Agency Type,Agency Name}  {Agency Name,State}  --> Agency Code
 {Record ID}  {Agency Code}  --> Agency Name
 {Record ID}  {Agency Code}  {Agency Name}  --> Agency Type
 {Record ID}  {Agency Code}  {Agency Name}  --> City
 {Record ID}  {Agency Name,City}  {Agency Code}  {Agency Type,Agency Name}  {Agency Type,Year,City}  {Year,Month,City,Incident}  {Year,Agency Name}  {Year,Crime Type,City,Month}  --> State
 {Record ID}  --> Year
 {Record ID}  --> Month
 {Record ID}  --> Incident
 {Record ID}  {Agency Code}  {Agency Name}  {Incident}  {State}  {City}  {Year}  {Agency Type}  {Month}  --> Crime Type


In [6]:
groupings = an.normalize_dependencies(deps_approx)
for grp in groupings:
    print('\n~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~\n')
    print(grp)


~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> Record ID
 {Record ID}  --> Agency Name
 {Record ID}  --> Year
 {Record ID}  --> Month
 {Record ID}  --> Incident

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 {Agency Code}  --> Agency Name
 {Agency Name}  {Agency Name}  {Agency Name}  --> Agency Code
 {Agency Code}  {Agency Name}  --> Agency Type
 {Agency Code}  {Agency Name}  --> City
 {Agency Name}  {Agency Code}  {Agency Name}  {Agency Name}  --> State

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> State
 {State}  --> Crime Type
